<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных соревнования. Знакомиться будем на [данных](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) Scikit-learn по новостям, сначала в режиме бинарной классификации, затем – в многоклассовом режиме. Затем будем классифицировать рецензии к фильмам с сайта IMDB. Наконец, применим Vowpal Wabbit к данным по веб-сессиям. Материала немало, но Vowpal Wabbit того стоит!

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [133]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = ('data')

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


In [6]:
train_df_400.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182793 entries, 1 to 182793
Data columns (total 21 columns):
site1      182793 non-null int64
time1      182793 non-null object
site2      181175 non-null float64
time2      181175 non-null object
site3      179441 non-null float64
time3      179441 non-null object
site4      178054 non-null float64
time4      178054 non-null object
site5      176653 non-null float64
time5      176653 non-null object
site6      175268 non-null float64
time6      175268 non-null object
site7      173960 non-null float64
time7      173960 non-null object
site8      172738 non-null float64
time8      172738 non-null object
site9      171437 non-null float64
time9      171437 non-null object
site10     170247 non-null float64
time10     170247 non-null object
user_id    182793 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 30.7+ MB


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [7]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [8]:
y = train_df_400['user_id']
class_encoder = LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем этим моделям нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [20]:
def dropcols_coo(C, idx_to_drop):
    idx_to_drop = np.unique(idx_to_drop)
    keep = ~np.in1d(C.col, idx_to_drop)
    C.data, C.row, C.col = C.data[keep], C.row[keep], C.col[keep]
    C.col -= idx_to_drop.searchsorted(C.col)    # decrement column indices
    C._shape = (C.shape[0], C.shape[1] - len(idx_to_drop))
    return C

In [21]:
def transform_to_csr_matrix(arr):
    rows = []
    columns = []
    data = []
    for i in range(arr.shape[0]):
        # Create dictionary {site: number of visits per session}
        unique, counts = np.unique(arr[i], return_counts=True)
        dic = dict(zip(unique, counts))
        rows.extend([i]*len(dic.keys()))
        columns.extend(dic.keys())
        data.extend(dic.values())
    
    # Sparse coo matrix
    arr_new_coo = coo_matrix((data, (rows, columns)))
    
    # Drop column with "zero" site and transform to csr
    return dropcols_coo(arr_new_coo, 0).tocsr()

In [22]:
sites = ['site' + str(i) for i in range(1, 11)]

In [174]:
train_test_df = pd.concat([train_df_400, test_df_400])
train_test_df_sites = train_test_df[sites].fillna(0).astype('int')
train_test_sparse = transform_to_csr_matrix(train_test_df_sites.values)

In [175]:
X_train_sparse = train_test_sparse[:train_df.shape[0]]
X_test_sparse = train_test_sparse[train_df.shape[0]:]

In [176]:
X_train_sparse.shape

(182793, 36656)

In [177]:
X_test_sparse.shape

(46473, 36656)

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [178]:
train_df = train_df_400[sites].fillna(0).astype(np.int32)
test_df = test_df_400[sites].fillna(0).astype(np.int32)

In [179]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :].fillna(0).astype(np.int32)
valid_df = train_df_400[sites].iloc[train_share:, :].fillna(0).astype(np.int32)
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [180]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [181]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    with open(os.path.join(PATH_TO_DATA, out_file), 'w') as out_vw:
        for i, row in enumerate(np.nditer(X, flags=['external_loop'], order='C')):
            arr_str = ' '.join(list(map(str, row)))
            if y is None:
                label = 1
            else:
                label = y[i]
            vw_str = '{} | {}'.format(label, arr_str)
            out_vw.write(vw_str + '\n')

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [182]:
%%time
arrays_to_vw(train_df_part.values, y_train_part_for_vw, out_file='train_part.vw')
arrays_to_vw(valid_df.values, y_valid_for_vw, out_file='valid.vw')
arrays_to_vw(train_df.values, y_for_vw, out_file='train.vw')
arrays_to_vw(test_df.values, out_file='test.vw')

CPU times: user 12.5 s, sys: 47.8 ms, total: 12.6 s
Wall time: 12.8 s


**Результат должен получиться таким.**

In [183]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [184]:
!head -3  $PATH_TO_DATA/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [185]:
!head -3 $PATH_TO_DATA/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbit на выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [186]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [187]:
%%time
!vw -d data/train_part.vw \
--passes 3 \
--oaa 400 \
-c \
-k \
-b 26 \
--random_seed 17 \
-f data/model.vw \
--quiet

CPU times: user 581 ms, sys: 108 ms, total: 689 ms
Wall time: 27.7 s


**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [188]:
%%time
!vw -i data/model.vw -t -d data/valid.vw \
-p vw_valid_pred.txt \
--quiet

CPU times: user 43.1 ms, sys: 35.7 ms, total: 78.8 ms
Wall time: 2.25 s


**Считайте прогнозы *data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [189]:
with open('vw_valid_pred.txt') as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]
    
accuracy_score(y_valid_for_vw, valid_prediction)

0.34541741128414605

In [190]:
y_valid_for_vw

array([  4, 160, 312, ..., 254,  91, 256])

In [191]:
np.array(valid_prediction)

array([188, 220, 364, ..., 118, 318, 125])

**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться долго (у меня – 15 минут) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1.**

In [142]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [143]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

/home/maxis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 43min 37s, sys: 15.7 s, total: 43min 53s
Wall time: 21min 57s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [144]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

CPU times: user 1min 23s, sys: 21.2 s, total: 1min 44s
Wall time: 27.2 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

**Какими получаются доли правильных ответов на отложенной выборке для Vowpal Wabbit, SGD и логистической регрессии? Запишите 3 числа в файл *answer6_1.txt* (в указанном порядке, сначала VW, потом SGD и logit) через пробел, округлив до 3 знаков после запятой.**

In [149]:
vw_valid_acc = accuracy_score(y_valid_for_vw, valid_prediction)
sgd_valid_acc = accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse))
logit_valid_acc = accuracy_score(y_valid, logit.predict(X_valid_sparse))

In [150]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [151]:
write_answer_to_file("{}  {} {}".format(round(vw_valid_acc, 3), round(sgd_valid_acc, 3),
                                     round(logit_valid_acc, 3)),  
                     'answer6_1.txt')

# 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [192]:
%%time
!vw -d data/train.vw \
--passes 3 \
--oaa 400 \
-c \
-k \
-b 26 \
--random_seed 17 \
-f data/model.vw \
--quiet

CPU times: user 829 ms, sys: 89.2 ms, total: 919 ms
Wall time: 35.5 s


**Сделайте прогноз для тестовой выборки.**

In [193]:
%%time
!vw -i data/model.vw -t -d data/test.vw \
-p vw_test_pred.txt \
--quiet

CPU times: user 41.5 ms, sys: 28 ms, total: 69.5 ms
Wall time: 1.95 s


In [194]:
with open('vw_test_pred.txt') as pred_file:
    vw_pred = np.array([int(label) for label in pred_file.readlines()])
    
vw_pred = class_encoder.inverse_transform(vw_pred - 1)

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [195]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [196]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [199]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [200]:
%%time
logit.fit(X_train_sparse, y)

/home/maxis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 1h 11min 11s, sys: 23.1 s, total: 1h 11min 34s
Wall time: 35min 47s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [202]:
%%time
sgd_logit.fit(X_train_sparse, y)

CPU times: user 1min 55s, sys: 21.3 s, total: 2min 16s
Wall time: 34.5 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [203]:
logit_test_pred = logit.predict(X_test_sparse)
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse)

In [204]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

**Какими получаются доли правильных ответов  Vowpal Wabbit, SGD и логистической регрессии на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4) соревнования? Запишите 3 числа в файл *answer6_2.txt* (в указанном порядке, сначала VW, потом SGD и logit) через пробел, округлив до 3 знаков после запятой.**

In [206]:
write_answer_to_file(' '.join(map(str, [0.188, 0.175, 0.199])), 'answer6_2.txt')

**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать и описать результаты в проекте. Не перепутайте! 

## Пути улучшения

Что еще можно попробовать в [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2):
 - Использовать ранее построенные признаки для улучшения модели
 - Настроить параметры Vowpal Wabbit с hyperopt
 - Попробовать TF-IDF и n-граммы

На следующей, заключительной, неделе мы оформим всю работу над проектом в виде одного файла (`html`, `pdf` или `ipynb`) и будем проверять проекты друг друга.